# Data Mining Project
### Giorgio Donati, g.donati24@studenti.unipi.it
### Pietro Francaviglia, p.francaviglia1@studenti.unipi.it
#### A.Y. 2021-2022

## Libraries and datasets

In [31]:
from sklearn.svm import SVC

from t3_constants import *
from t3_utility import *

# Support Vector Classifier (Polynomial)
## Training

In [32]:
with open(f'{PICKLE_FOLDER}{TRAIN}{X}.pkl', 'rb') as f:
    train_x = pickle.load(f)
with open(f'{PICKLE_FOLDER}{TRAIN}{Y}.pkl', 'rb') as f:
    train_y = pickle.load(f)

In [33]:
# parameters dictionary for Decision Tree grid search
param_d = {
    KFF: [0.5, 1, 1.5],
    CPRM: [0.1, 0.5, 1, 5, 10, 50, 100],
    DEG: [2, 3, 4],
    KRL: ['poly']
}
# iteration on parameters to collect results
grid = itertools.product(*[param_d[key] for key in sorted(param_d)])
results = []
for c_param, coefficient, degree, kernel in grid:
    model=SVC(kernel=kernel, degree=degree, coef0=coefficient, C=c_param)
    mean_val_score, std_val_score = cross_validation_summary(model, train_x.values, train_y.values)
    res = c_param, coefficient, degree, kernel, mean_val_score, std_val_score
    results.append(res)

# print top 5 combination results
result_df = pd.DataFrame(results, columns=sorted(param_d) + [MVS, SVS])
result_df.sort_values(MVS, ascending=False).head(8)

,c_param,coefficient,degree,kernel,mean_val_score,std_val_score
27,5.0,0.5,2,poly,0.939263,0.020050
14,0.5,1.0,4,poly,0.937663,0.018598
28,5.0,0.5,3,poly,0.937663,0.019927
34,5.0,1.5,3,poly,0.937651,0.021179
23,1.0,1.0,4,poly,0.937651,0.024539
51,50.0,1.5,2,poly,0.937638,0.025557
48,50.0,1.0,2,poly,0.937638,0.025557
25,1.0,1.5,3,poly,0.936063,0.020935


In [34]:
result_df.groupby(CPRM)[MVS, SVS].mean().sort_values(MVS, ascending=False)

/var/folders/qc/cq4vvhlx6msd3j5kh8vlw_cm0000gn/T/ipykernel_21062/2357791576.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result_df.groupby(CPRM)[MVS, SVS].mean().sort_values(MVS, ascending=False)


,mean_val_score,std_val_score
c_param,,
5.0,0.933568,0.022981
10.0,0.927173,0.024476
50.0,0.927156,0.030234
1.0,0.924876,0.021080
100.0,0.922363,0.029795
0.5,0.919385,0.020576
0.1,0.891707,0.026550


In [35]:
result_df.groupby(KFF)[MVS, SVS].mean().sort_values(MVS, ascending=False)

/var/folders/qc/cq4vvhlx6msd3j5kh8vlw_cm0000gn/T/ipykernel_21062/879860663.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result_df.groupby(KFF)[MVS, SVS].mean().sort_values(MVS, ascending=False)


,mean_val_score,std_val_score
coefficient,,
1.0,0.923194,0.025729
1.5,0.923192,0.023586
0.5,0.916284,0.025981


In [36]:
result_df.groupby(DEG)[MVS, SVS].mean().sort_values(MVS, ascending=False)

/var/folders/qc/cq4vvhlx6msd3j5kh8vlw_cm0000gn/T/ipykernel_21062/3676944463.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  result_df.groupby(DEG)[MVS, SVS].mean().sort_values(MVS, ascending=False)


,mean_val_score,std_val_score
degree,,
3,0.922821,0.024337
4,0.920758,0.027467
2,0.919089,0.023492


### Test Model

In [37]:
best_result = result_df.sort_values(MVS).tail(1)

# best model
test_model = SVC(
    kernel=best_result[KRL].values[0],
    degree=best_result[DEG].values[0],
    coef0=best_result[KFF].values[0],
    C=best_result[CPRM].values[0]
)
_ = test_model.fit(train_x, train_y)

## Testing

In [38]:
with open(f'{PICKLE_FOLDER}{TEST}{X}.pkl', 'rb') as f:
    test_x = pickle.load(f)
with open(f'{PICKLE_FOLDER}{TEST}{Y}.pkl', 'rb') as f:
    test_y = pickle.load(f)

### Score

In [39]:
test_model.score(test_x, test_y)

0.9490445859872612